<h1><center> Projet Machine Learning avec Python : </center></h1>
<h1><center> Prédiction des incidents  à San Francisco </center></h1>



<h1><center> Partie 2 : Prédiction du nombre d'appels par heure, date et quartier <center></h1>

In [1]:
import os

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import seaborn as sns
sns.set(rc={'figure.figsize':(11, 4)})

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score

from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier

import tensorflow as tf
from tensorflow import keras

# Préparation de notre base de données pour la prédiction du nombre d'appels par heure, date et quartier

In [2]:
path = 'C:/Users/Allard/Desktop/' # Change to path to data
os.chdir(path)

In [3]:
df = pd.read_csv('Fire_Department_Calls_for_Service_reduced_V3.csv')

C:\Users\Allard\anaconda3\envs\geopandas\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (6) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


Nombre d'appels par heure, date et lieu

In [4]:
count_test = df.groupby(['year', 'month', 'day', 'hour', 'weekday', 'Neighborhooods - Analysis Boundaries']).size().reset_index(name='count')

Création d'un dataframe contenant toutes les heures entre le 01/01/2012 et le 31/12/2020

In [5]:
df_2012_2020 = pd.DataFrame(
        {'Received DtTm': pd.date_range('2012-01-01', '2021-01-01', freq='1H', closed='left')}
     )

Produit cartésien entre les heures/dates et les 41 quartiers de San Francisco

In [6]:
df_cartesian = df_2012_2020.merge(pd.DataFrame(df['Neighborhooods - Analysis Boundaries'].unique(), columns=["Neighborhooods - Analysis Boundaries"]), how='cross')

df_cartesian['year'] = df_cartesian['Received DtTm'].dt.year
df_cartesian['month'] = df_cartesian['Received DtTm'].dt.month
df_cartesian['day'] = df_cartesian['Received DtTm'].dt.day
df_cartesian['hour'] = df_cartesian['Received DtTm'].dt.hour
df_cartesian["weekday"] = df_cartesian['Received DtTm'].dt.day_name()


df_cartesian = df_cartesian.drop(columns='Received DtTm')

Fusion de nos 2 bases de données afin d'avoir pour chaque heure, date et lieu, le nombre d'appels (qui peut donc être nul)

In [7]:
result = pd.merge(df_cartesian, count_test, on=["year", "month", "day", "hour", "weekday", "Neighborhooods - Analysis Boundaries"], how='left')

result = result.fillna(0)

result

,Neighborhooods - Analysis Boundaries,year,month,day,hour,weekday,count
0,Financial District/South Beach,2012,1,1,0,Sunday,23.0
1,Western Addition,2012,1,1,0,Sunday,3.0
2,Hayes Valley,2012,1,1,0,Sunday,4.0
3,South of Market,2012,1,1,0,Sunday,4.0
4,Treasure Island,2012,1,1,0,Sunday,2.0
...,...,...,...,...,...,...,...
3235387,Presidio Heights,2020,12,31,23,Thursday,0.0
3235388,Seacliff,2020,12,31,23,Thursday,0.0
3235389,Glen Park,2020,12,31,23,Thursday,0.0
3235390,McLaren Park,2020,12,31,23,Thursday,0.0


On essaie de predire si le nombre d'appel est égale à 0, 1, 2, 3, ou alors supérieur ou égale à 4

In [8]:
result.loc[result['count'] >= 4, 'count'] = 4

On encode les colonnes textuelles en integer

In [9]:
result['Neighborhooods - Analysis Boundaries'] = result['Neighborhooods - Analysis Boundaries'].astype('category')
result['Neighborhooods - Analysis Boundaries'] = result['Neighborhooods - Analysis Boundaries'].cat.codes

result['weekday'] = result['weekday'].astype('category')
result['weekday'] = result['weekday'].cat.codes

Notre base de données est très grande pour accélérer les algo de ML nous limiterons nos données aux années 2017-2018-2019

In [10]:
result = result[(result['year']>2016) & (result['year']<2020)]

In [11]:
result

,Neighborhooods - Analysis Boundaries,year,month,day,hour,weekday,count
1797768,5,2017,1,1,0,3,4.0
1797769,40,2017,1,1,0,3,0.0
1797770,9,2017,1,1,0,3,1.0
1797771,33,2017,1,1,0,3,4.0
1797772,36,2017,1,1,0,3,0.0
...,...,...,...,...,...,...,...
2875243,30,2019,12,31,23,5,0.0
2875244,32,2019,12,31,23,5,0.0
2875245,6,2019,12,31,23,5,0.0
2875246,17,2019,12,31,23,5,0.0


In [12]:
y = result["count"]
X = result.drop(columns = ["count","year"], axis=1)

Regardons la répartition du nombre d'appels

In [13]:
y.groupby(y).size()

count
0.0    785011
1.0    200287
2.0     56414
3.0     20573
4.0     15195
Name: count, dtype: int64

In [14]:
for i in range (5):
    print('Le pourcentage de ' + str(i) + ' dans nos outputs est: {}%'.format(round(y.groupby(y).size()[i]/len(y)*100,1)))

Le pourcentage de 0 dans nos outputs est: 72.9%
Le pourcentage de 1 dans nos outputs est: 18.6%
Le pourcentage de 2 dans nos outputs est: 5.2%
Le pourcentage de 3 dans nos outputs est: 1.9%
Le pourcentage de 4 dans nos outputs est: 1.4%


On peut voir que la classe 0 est sur-représenté avec environ 73%. Ensuite il va falloir faire attention dans nos prédiction de regarder si les autres classes sont bien prédites car il est très facile d'avoir une accuracy de 73% sur notre modèle.

# PREDICTION AVEC RN

Pour notre premier modèle, nous allons utiliser un réseau de neurones (RN) avec 2 couches cachées constituées de 50 neurones. La couche de sortie sera constituée de 5 neurones pour nos 5 classes (0, 1, 2, 3, 4) avec une fonction d'activation softmax qui nous permettra d'obtenir la probabilité de chaque classe est d'en déduire la prédiction en prenant la probabilité la plus forte.

Dans premier temps, nous préparons notre jeu de données en effectuer une réduction d'echelle (MinMax) afin que le RN converge plus rapidement. Puis nous effecture une transformation OneHotEncoder pour créer 5 colonnes en sortie afin d'obtenir la probabilité de chaque classe.

In [15]:
from sklearn.preprocessing import OneHotEncoder
from numpy import array
from numpy import argmax
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
X_reduced = scaler.fit_transform(X)

# one hot encode
y_reduced = tf.keras.utils.to_categorical(y)

X_train, X_test, y_train, y_test = train_test_split(X_reduced, y_reduced, test_size=0.2)


In [16]:
model_soft = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=[5]),
    tf.keras.layers.Dense(20, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(20, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(5, activation='softmax')
])

model_soft.compile(
  optimizer='adam',
  loss="categorical_crossentropy",
  metrics=[tf.keras.metrics.CategoricalAccuracy()])

In [17]:
history_soft = model_soft.fit(X_train, y_train, epochs=10, batch_size=128, validation_split=0.2)

Epoch 1/10
5388/5388 [==============================] - 9s 2ms/step - loss: 0.8406 - categorical_accuracy: 0.7260 - val_loss: 0.8143 - val_categorical_accuracy: 0.7301
Epoch 2/10
5388/5388 [==============================] - 8s 2ms/step - loss: 0.8175 - categorical_accuracy: 0.7283 - val_loss: 0.8050 - val_categorical_accuracy: 0.7301
Epoch 3/10
5388/5388 [==============================] - 8s 2ms/step - loss: 0.8044 - categorical_accuracy: 0.7283 - val_loss: 0.7857 - val_categorical_accuracy: 0.7301
Epoch 4/10
5388/5388 [==============================] - 8s 2ms/step - loss: 0.7923 - categorical_accuracy: 0.7283 - val_loss: 0.7749 - val_categorical_accuracy: 0.7301
Epoch 5/10
5388/5388 [==============================] - 8s 2ms/step - loss: 0.7863 - categorical_accuracy: 0.7283 - val_loss: 0.7724 - val_categorical_accuracy: 0.7301
Epoch 6/10
5388/5388 [==============================] - 8s 2ms/step - loss: 0.7827 - categorical_accuracy: 0.7283 - val_loss: 0.7691 - val_categorical_accuracy:

In [18]:
model_soft.evaluate(X_test, y_test)

6735/6735 [==============================] - 6s 822us/step - loss: 0.7669 - categorical_accuracy: 0.7283


[0.7669259905815125, 0.7283430099487305]

Notre modèle est évalué avec une performance d'accuracy (72.8%), qui est un score plutôt convenable au premier abord mais qui ressemble étrangement au même pourcentage de la classe 0. 

Regardons de plus près les prédictions faites:

In [19]:
y_pred = model_soft.predict(X_test)

In [20]:
yhat = np.argmax(y_pred, axis=1)

In [21]:
yhat[:100]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int64)

In [22]:
y_true = argmax(y_test, axis=1)

In [23]:
y_true[:100]

array([0, 1, 0, 1, 0, 0, 0, 0, 2, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 1, 0, 0, 1, 2, 0, 0, 1, 0, 1, 2, 0, 0, 0, 4, 3, 0, 0, 0, 0, 2,
       0, 1, 0, 2, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 1,
       0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 1,
       0, 1, 2, 0, 0, 4, 0, 1, 0, 0, 1, 0], dtype=int64)

In [24]:
sum(yhat==0)

215496

In [25]:
yhat.shape

(215496,)

Nous pouvons nous apercevoir que notre modèle prédit toujours 0. Etant donné que la classe 0 est majoriatire (environ 73%) nous avons une accuracy correcte mais qui ne correspond pas du tout à ce que nous souhaitons. Il nous faut plutôt regarder le recall ou le F1-score qui prennent en considération les False Négative (FN) en particulier dans les classes 1, 2, 3 et 4.

In [26]:
f1_score(y_true, yhat, average='weighted')

0.6138636936764467

In [27]:
f1_score(y_true, yhat, average=None)

array([0.84282228, 0.        , 0.        , 0.        , 0.        ])

Nous allons partir sur des techniques de ML plutôt qu'essayer de paramétrer notre RN.

# PREDICTION AVEC RF

In [28]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [29]:
%%time
#Prédiction avec les forêts aléatoires
rf=RandomForestClassifier()
parameters={'n_estimators': [25, 50]}  # defining parameter space

clf_rf= GridSearchCV(estimator=rf, param_grid=parameters, cv= 3)
clf_rf.fit(X_train, y_train)

print('Returned hyperparameter: {}'.format(clf_rf.best_params_))
print('Best Balanced accuracy in train is: {}'.format(clf_rf.best_score_))
print('Classification accuracy on test is: {}'.format(clf_rf.score(X_test, y_test)))

Returned hyperparameter: {'n_estimators': 50}
Best Balanced accuracy in train is: 0.7075676578683595
Classification accuracy on test is: 0.7064446671863979
Wall time: 7min 33s


In [30]:
y_hat_rf = clf_rf.predict(X_test)

Nous allons observer la matrice de confusion qui nous permet de vérifier rapidement si notre modèle classifie correctement ou non.

In [31]:
confusion_matrix(y_test, y_hat_rf)

array([[143722,  10831,   1698,    470,    262],
       [ 30829,   6059,   1784,    703,    571],
       [  6128,   2701,   1271,    692,    627],
       [  1231,   1057,    787,    473,    526],
       [   425,    717,    708,    513,    711]], dtype=int64)

Puis maintenant regardons le F1-score qui est une valeur quantifiant la performance de notre modèle.

In [32]:
f1_score(y_test, y_hat_rf, average='weighted')

0.6674653616878513

In [33]:
f1_score(y_test, y_hat_rf, average=None)

array([0.84712276, 0.19764806, 0.14388408, 0.1366065 , 0.24640444])

Nous pouvons voir que la classe 0 (aucun appel dans l'heure) est plutôt correctement classifié alors que les autres classes (c'est à dire au moins appel) ne dépasse pas un score F1 de 0.24 ce qui est relativement faible. Ceci peut s'expliquer par le fait que la classe 0 est sur-représenté et que nos variables (inputs) n'explique pas suffisamment qu'il y ait des appels ou non.

# PREDICTION AVEC KNN

In [34]:
%%time
#Prédiction avec les plus proches voisins
knn=KNeighborsClassifier()
parameters={'n_neighbors': [2, 3, 4, 5]}  # defining parameter space

clf_knn = GridSearchCV(estimator=knn, param_grid=parameters, cv= 3)
clf_knn.fit(X_train, y_train)

print('Returned hyperparameter: {}'.format(clf_knn.best_params_))
print('Best Balanced accuracy in train is: {}'.format(clf_knn.best_score_))
print('Classification accuracy on test is: {}'.format(clf_knn.score(X_test, y_test)))

Returned hyperparameter: {'n_neighbors': 4}
Best Balanced accuracy in train is: 0.7108612224820878
Classification accuracy on test is: 0.7123473289527416
Wall time: 5min 2s


In [35]:
y_hat_knn = clf_knn.predict(X_test)

In [36]:
confusion_matrix(y_test, y_hat_knn)

array([[148466,   7193,   1038,    173,    113],
       [ 34673,   3982,    812,    265,    214],
       [  8561,   1800,    582,    249,    227],
       [  2593,    728,    381,    169,    203],
       [  1632,    553,    380,    200,    309]], dtype=int64)

In [37]:
f1_score(y_test, y_hat_knn, average='weighted')

0.6477597924624308

In [38]:
f1_score(y_test, y_hat_knn, average=None)

array([0.84138642, 0.14693185, 0.07966055, 0.06588694, 0.14927536])

Les résultats sont encore moins bons qu'avec un Random Forest. Nous devons essayer de trouver un moyen d'améliorer notre modèle en ajoutant de nouvelles variables telles que la météo ou les jours fériés au USA qui pourront peut-être expliquer le nombre d'appels par heure.

# Amélioration du modèle en ajoutant la météo

On peut penser que la météo peut avoir un rôle à jouer car si le temps est ensoleillé les gens seront surement plus à même de sortir de chez eux et donc se avoir un accident par exemple ou bien qu'il y ait un cambriolage etc...
Regardons si la météo joue un rôle dans le nombre d'appels 

In [39]:
weather = pd.read_csv('weather_description.csv')

In [40]:
weather.head()

,datetime,Vancouver,Portland,San Francisco,Seattle,Los Angeles,San Diego,Las Vegas,Phoenix,Albuquerque,...,Philadelphia,New York,Montreal,Boston,Beersheba,Tel Aviv District,Eilat,Haifa,Nahariyya,Jerusalem
0,2012-10-01 12:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,haze,NaN,NaN,NaN
1,2012-10-01 13:00:00,mist,scattered clouds,light rain,sky is clear,mist,sky is clear,sky is clear,sky is clear,sky is clear,...,broken clouds,few clouds,overcast clouds,sky is clear,sky is clear,sky is clear,haze,sky is clear,sky is clear,sky is clear
2,2012-10-01 14:00:00,broken clouds,scattered clouds,sky is clear,sky is clear,sky is clear,sky is clear,sky is clear,sky is clear,sky is clear,...,broken clouds,few clouds,sky is clear,few clouds,sky is clear,sky is clear,broken clouds,overcast clouds,sky is clear,overcast clouds
3,2012-10-01 15:00:00,broken clouds,scattered clouds,sky is clear,sky is clear,sky is clear,sky is clear,sky is clear,sky is clear,sky is clear,...,broken clouds,few clouds,sky is clear,few clouds,overcast clouds,sky is clear,broken clouds,overcast clouds,overcast clouds,overcast clouds
4,2012-10-01 16:00:00,broken clouds,scattered clouds,sky is clear,sky is clear,sky is clear,sky is clear,sky is clear,sky is clear,sky is clear,...,broken clouds,few clouds,sky is clear,few clouds,overcast clouds,sky is clear,broken clouds,overcast clouds,overcast clouds,overcast clouds


On ne sélectionne que la météo pour San Francisco

In [41]:
weather=weather[['datetime', 'San Francisco']]

In [42]:
weather=weather.dropna()
weather['datetime'] = pd.to_datetime(weather['datetime'], infer_datetime_format=True)


In [43]:
weather['year'] = weather['datetime'].dt.year
weather['month'] = weather['datetime'].dt.month
weather['day'] = weather['datetime'].dt.day
weather['hour'] = weather['datetime'].dt.hour

weather["weekday"] = weather['datetime'].dt.day_name()

In [44]:
weather.groupby('year').size()

year
2012    2195
2013    8760
2014    8760
2015    8760
2016    8784
2017    7201
dtype: int64

D'après la commande précédente seule les années 2013 à 2016 sont complètes. Ainsi nous allons conserver les années 2014-2015-2016 (3 ans d'historique)

In [45]:
weather = weather[(weather['year']>2013) & (weather['year']<2017)]

In [46]:
weather = weather.drop(columns='datetime')

In [47]:
result_weather = pd.merge(df_cartesian, weather, on=["year", "month", "day", "hour", "weekday"])

In [48]:
result_weather.rename(columns={'San Francisco': 'weather'}, inplace=True)

Nous fusionnons notre base de données avec la base de données initiale contenant le nombre d'appels par heure et par quartier

In [49]:
result_weather_2 = pd.merge(result_weather, count_test, on=["year", "month", "day", "hour", "weekday", "Neighborhooods - Analysis Boundaries"], how='left')

result_weather_2 = result_weather_2.fillna(0)

Encore une fois au delà de 3 appels nous considérons une seule et même classe du nombre d'appels

In [50]:
result_weather_2.loc[result_weather_2['count'] >= 4, 'count'] = 4

In [51]:
result_weather_2['Neighborhooods - Analysis Boundaries'] = result_weather_2['Neighborhooods - Analysis Boundaries'].astype('category')
result_weather_2['Neighborhooods - Analysis Boundaries'] = result_weather_2['Neighborhooods - Analysis Boundaries'].cat.codes

result_weather_2['weekday'] = result_weather_2['weekday'].astype('category')
result_weather_2['weekday'] = result_weather_2['weekday'].cat.codes

result_weather_2['weather'] = result_weather_2['weather'].astype('category')
result_weather_2['weather'] = result_weather_2['weather'].cat.codes

Nous splittons nos inputs et output

In [52]:
y_weather = result_weather_2["count"]
X_weather = result_weather_2.drop(columns = ["count","year"], axis=1)

In [53]:
y_weather.groupby(y_weather).size()

count
0.0    807444
1.0    191082
2.0     50835
3.0     17722
4.0     11381
Name: count, dtype: int64

Modèle avec un RF

In [54]:
%%time

X_train_weather, X_test_weather, y_train_weather, y_test_weather = train_test_split(X_weather, y_weather, test_size=0.2)

#Prédiction avec les forêts aléatoires
rf=RandomForestClassifier()
parameters={'n_estimators': [25, 50]}  # defining parameter space

clf_rf= GridSearchCV(estimator=rf, param_grid=parameters, cv= 3)
clf_rf.fit(X_train_weather, y_train_weather)

print('Returned hyperparameter: {}'.format(clf_rf.best_params_))
print('Best Balanced accuracy in train is: {}'.format(clf_rf.best_score_))
print('Classification accuracy on test is: {}'.format(clf_rf.score(X_test_weather, y_test_weather)))

Returned hyperparameter: {'n_estimators': 50}
Best Balanced accuracy in train is: 0.7289999308411574
Classification accuracy on test is: 0.7290408126364787
Wall time: 8min 4s


In [55]:
y_hat_rf_w = clf_rf.predict(X_test_weather)

In [56]:
confusion_matrix(y_test_weather, y_hat_rf_w)

array([[150203,   9569,   1349,    322,    184],
       [ 30425,   5338,   1479,    488,    386],
       [  5833,   2417,   1113,    480,    419],
       [  1293,   1038,    594,    289,    300],
       [   450,    617,    513,    288,    306]], dtype=int64)

In [57]:
f1_score(y_test_weather, y_hat_rf_w, average='weighted')

0.6868170201206522

In [58]:
f1_score(y_test_weather, y_hat_rf_w, average=None)

array([0.8587175 , 0.1869866 , 0.14539517, 0.10741498, 0.16237729])

# Ajout des jours fériés

Nous avons vu que la météo n'avait pas d'impact sur notre modèle de prévision. Nous nous intéressons maintenant si les jours fériés peuvent avoir un impact sur le nombre d'appels

In [59]:
dayoff = pd.read_csv('usholidays.csv')

In [60]:
dayoff.columns

Index(['Unnamed: 0', 'Date', 'Holiday'], dtype='object')

In [61]:
dayoff=dayoff.dropna()
dayoff['Date'] = pd.to_datetime(dayoff['Date'], infer_datetime_format=True)

dayoff['year'] = dayoff['Date'].dt.year
dayoff['month'] = dayoff['Date'].dt.month
dayoff['day'] = dayoff['Date'].dt.day
dayoff["weekday"] = dayoff['Date'].dt.day_name()

#notre base de données principale contient uniquement les jours entre le 01/01/2012 et 31/12/2020
dayoff = dayoff[dayoff['year']>2011]

#1 si c'est un jour férié 1 et par la suite 0 si c'est pas un jour férié
dayoff['Date'] = 1
dayoff.rename(columns={'Date': 'public holiday'}, inplace=True)
dayoff = dayoff.drop(columns=["Unnamed: 0", "Holiday"])

dayoff.groupby('year').size()

year
2012    10
2013    10
2014    10
2015    10
2016    10
2017    10
2018    10
2019    10
2020    10
dtype: int64

In [62]:
result_dayoff = pd.merge(count_test, dayoff, on=["year", "month", "day", "weekday"], how='left')
result_dayoff = result_dayoff.fillna(0)

In [63]:
result_dayoff2 = pd.merge(df_cartesian, result_dayoff, on=["year", "month", "day", "hour", "weekday", "Neighborhooods - Analysis Boundaries"], how='left')
result_dayoff2 = result_dayoff2.fillna(0)

In [64]:
result_dayoff2.loc[result_dayoff2['count'] >= 4, 'count'] = 4

result_dayoff2['Neighborhooods - Analysis Boundaries'] = result_dayoff2['Neighborhooods - Analysis Boundaries'].astype('category')
result_dayoff2['Neighborhooods - Analysis Boundaries'] = result_dayoff2['Neighborhooods - Analysis Boundaries'].cat.codes

result_dayoff2['weekday'] = result_dayoff2['weekday'].astype('category')
result_dayoff2['weekday'] = result_dayoff2['weekday'].cat.codes

result_dayoff2 = result_dayoff2[(result_dayoff2['year']>2016) & (result_dayoff2['year']<2020)]

In [65]:
y_dayoff = result_dayoff2["count"]
X_dayoff = result_dayoff2.drop(columns = ["count","year"], axis=1)

A nouveau, notre modèle se base sur un RF

In [66]:
%%time

X_train_do, X_test_do, y_train_do, y_test_do = train_test_split(X_dayoff, y_dayoff, test_size=0.2)

#Prédiction avec les forêts aléatoires
rf=RandomForestClassifier()
parameters={'n_estimators': [25, 50]}  # defining parameter space

clf_rf= GridSearchCV(estimator=rf, param_grid=parameters, cv= 3)
clf_rf.fit(X_train_do, y_train_do)

print('Returned hyperparameter: {}'.format(clf_rf.best_params_))
print('Best Balanced accuracy in train is: {}'.format(clf_rf.best_score_))
print('Classification accuracy on test is: {}'.format(clf_rf.score(X_test_do, y_test_do)))

Returned hyperparameter: {'n_estimators': 50}
Best Balanced accuracy in train is: 0.7139297249136876
Classification accuracy on test is: 0.7137905111927831
Wall time: 7min 12s


In [67]:
y_hat_rf_do = clf_rf.predict(X_test_do)

In [68]:
confusion_matrix(y_test_do, y_hat_rf_do)

array([[144685,  10055,   1504,    468,    257],
       [ 30573,   6651,   1762,    655,    551],
       [  5915,   2733,   1296,    647,    570],
       [  1280,   1107,    767,    437,    501],
       [   473,    729,    682,    448,    750]], dtype=int64)

In [69]:
f1_score(y_test_do, y_hat_rf_do, average='weighted')

0.6745270689889429

In [70]:
f1_score(y_test_do, y_hat_rf_do, average=None)

array([0.85135115, 0.2164088 , 0.1509434 , 0.12953905, 0.26265102])